In [28]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, Adadelta
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

from sklearn.metrics import accuracy_score

from getModel import getDiscriminator, getGenerator

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [34]:
if 1:
    def custom_loss(yTrue, yPred):
        return 100 * losses.mse(yTrue, yPred) + 0.0001 * sobelSELoss(yTrue, yPred)
    
    def expandedSobel(inputTensor):
        #this contains both X and Y sobel filters in the format (3,3,1,2)
        #size is 3 x 3, it considers 1 input channel and has two output channels: X and Y
        sobelFilter = K.variable([[[[1.,  1.]], [[0.,  2.]],[[-1.,  1.]]],
                              [[[2.,  0.]], [[0.,  0.]],[[-2.,  0.]]],
                              [[[1., -1.]], [[0., -2.]],[[-1., -1.]]]])
    
        #this considers data_format = 'channels_last'
        inputChannels = K.reshape(K.ones_like(inputTensor[0,0,0,:]),(1,1,-1,1))
        #if you're using 'channels_first', use inputTensor[0,:,0,0] above

        return sobelFilter * inputChannels

    
    def sobelSELoss(yTrue, yPred):

        #get the sobel filter repeated for each input channel
        filt = expandedSobel(yTrue)

        #calculate the sobel filters for yTrue and yPred
        #this generates twice the number of input channels 
        #a X and Y channel for each input channel
        sobelTrue = K.depthwise_conv2d(yTrue,filt)
        sobelPred = K.depthwise_conv2d(yPred,filt)

        #now you just apply the mse:
        return K.abs( K.sum(K.abs(sobelTrue)) - K.sum(K.abs(sobelPred)) )

In [35]:
def tranform(data):
    return (data - 0.5) * 2

class ContextEncoder():
    def __init__(self):

        self.img_rows = 128
        self.img_cols = 128
        
        self.channels = 1
        self.num_classes = 2
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        #optimizer = Adam(0.0002, 0.5) # Original one for minst
        optimizer = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
        #optimizer = RMSprop(lr=0.00005) # Trick from paper of WGAN

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates the missing
        # part of the image
        masked_img = Input(shape=self.img_shape)
        gen_missing = self.generator(masked_img)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines
        # if it is generated or if it is a real image
        valid = self.discriminator(gen_missing)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(masked_img , [gen_missing, valid])
        
        self.combined.compile(loss=[custom_loss, 'binary_crossentropy'], # Original loss from ContextEncoder
            loss_weights=[0.99, 0.01],
            optimizer=optimizer)
        

        
    
        
    def build_generator(self):
        
        model = getGenerator(self.img_shape, out_activation = 'tanh') # For range  -1 - 1
        
        X = Input(shape=self.img_shape)
        yhat = model(X)

        return Model(X, yhat)

    def build_discriminator(self):
        
        model = getDiscriminator(self.img_shape, out_activation = 'sigmoid') # For range  0 - 1
        
        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    
    def train_map(self, data, epochs, batch_size=128, sample_interval=50):
        
        (masked_imgs, missing_parts), (X_test, y_test) = data
        
        masked_imgs, missing_parts,X_test, y_test = [tranform(ds) for ds in [masked_imgs, missing_parts,
                                                                             X_test, y_test]]
        
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        y_test_bi = np.bool8(y_test > 0)  # -1 - 1
        #y_test_bi = np.bool8(y_test > 0.5)  #  0 - 1
        
        for epoch in range(epochs):
            
            idx = np.random.randint(0, masked_imgs.shape[0], batch_size)
            
            batch_masked_imgs = masked_imgs[idx]
            batch_missing_parts = missing_parts[idx]
            
            # Generate a batch of new images
            batch_gen_missing = self.generator.predict(batch_masked_imgs)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(batch_missing_parts, valid)
            d_loss_fake = self.discriminator.train_on_batch(batch_gen_missing, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(batch_masked_imgs, [batch_missing_parts, valid])          
            
            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:  
                
                # Plot the progress
                print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

                idx = [278, 451, 3263]
                #idx = np.random.randint(0, X_test.shape[0], 3)
                img_in, img_out = X_test[idx], y_test[idx]
                self.sample_maps(epoch, img_in, img_out)
                
                X_test_pred = self.predict(X_test)
                
                labels_pred = self.discriminator.predict(X_test_pred)
                
                print('Generated images who pass discriminator:', sum(labels_pred > 0.5), len(labels_pred))
                #print(np.max(X_test_pred), np.min(X_test_pred), np.max(y_test), np.min(y_test))
                
                X_test_pred_bi = np.bool8(X_test_pred > 0)    # -1 - 1
                #X_test_pred_bi = np.bool8(X_test_pred > 0.5) #  0 - 1
                
                accs = [accuracy_score(np.reshape(j, (128, 128)).flatten(), 
                                       np.reshape(i, (128, 128)).flatten()) for i, j in zip(X_test_pred_bi, y_test_bi)]
                print(np.mean(accs))
      
    
    def predict(self, img_in):
        
        gen_missing = self.generator.predict(img_in)
        
        return gen_missing
    
    def sample_maps(self, epoch, img_in, img_out):

        r, c = 3, 3
        
        gen_missing = self.generator.predict(img_in)

        imgs = 0.5 * img_in + 0.5
        masked_imgs = 0.5 * img_in + 0.5
        gen_missing = 0.5 * gen_missing + 0.5

        fig, axs = plt.subplots(r, c, figsize=(15,15))
        for i in range(c):
            axs[0,i].imshow(np.reshape(img_in[i, :,:], (self.img_rows, self.img_cols)))
            axs[0,i].axis('off')
            axs[1,i].imshow(np.reshape(img_out[i, :,:], (self.img_rows, self.img_cols)))
            axs[1,i].axis('off')
            axs[2,i].imshow(np.reshape(gen_missing[i, :,:], (self.img_rows, self.img_cols)))
            axs[2,i].axis('off')
        
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")

In [36]:
def load_dataset():
    
    scale = 25
    p_size_1 = 128 # Compared with 256, which larger may generate round corners
    trainPath = r"../tmp_data/data_feng/geb" + str(scale) +  "/"

    x_train_sim = np.load(trainPath + "x_train_sim.npy")
    y_train_sim = np.load(trainPath + "y_train_sim.npy")
    x_test_sim = np.load(trainPath + "x_test_sim.npy")
    y_test_sim = np.load(trainPath + "y_test_sim.npy")
    
    return (x_train_sim, y_train_sim), (x_test_sim, y_test_sim)

In [4]:
if __name__ == '__main__':
    
    data = load_dataset()
    #(_, _), (X_test, y_test) = data

In [37]:
    context_encoder = ContextEncoder()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_171 (Conv2D)          (None, 64, 64, 16)        416       
_________________________________________________________________
leaky_re_lu_129 (LeakyReLU)  (None, 64, 64, 16)        0         
_________________________________________________________________
batch_normalization_159 (Bat (None, 64, 64, 16)        64        
_________________________________________________________________
conv2d_172 (Conv2D)          (None, 32, 32, 32)        4640      
_________________________________________________________________
leaky_re_lu_130 (LeakyReLU)  (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_160 (Bat (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_173 (Conv2D)          (None, 16, 16, 64)        18496     
__________

In [38]:
    context_encoder.train_map(data, epochs=8000, batch_size=64, sample_interval=50)  

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.991020, acc: 52.34%] [G loss: 530.371155, mse: 535.717163]
Generated images who pass discriminator: [3341] 3528
0.697589554214
50 [D loss: 0.136599, acc: 96.88%] [G loss: 32.007515, mse: 32.268585]
Generated images who pass discriminator: [3503] 3528
0.925784675442
100 [D loss: 0.006517, acc: 100.00%] [G loss: 23.313082, mse: 23.458652]
Generated images who pass discriminator: [3528] 3528
0.930221920922
150 [D loss: 0.015011, acc: 100.00%] [G loss: 18.955936, mse: 19.079283]
Generated images who pass discriminator: [3528] 3528
0.927433731875
200 [D loss: 0.004585, acc: 100.00%] [G loss: 18.212379, mse: 18.277893]
Generated images who pass discriminator: [3528] 3528
0.936541481624
250 [D loss: 0.003642, acc: 100.00%] [G loss: 18.200161, mse: 18.283619]
Generated images who pass discriminator: [3528] 3528
0.948291752614
300 [D loss: 0.000304, acc: 100.00%] [G loss: 15.473019, mse: 15.509571]
Generated images who pass discriminator: [3528] 3528
0.956735399034
350 [D loss: 0.0

ResourceExhaustedError: OOM when allocating tensor with shape[64,48,128,128]
	 [[Node: training_10/Adadelta/gradients/model_22/sequential_16/batch_normalization_182/FusedBatchNorm_grad/FusedBatchNormGrad = FusedBatchNormGrad[T=DT_FLOAT, _class=["loc:@model_22/sequential_16/batch_normalization_182/FusedBatchNorm"], data_format="NHWC", epsilon=0.001, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_10/Adadelta/gradients/model_22/sequential_16/batch_normalization_182/cond/Switch_1_grad/cond_grad, model_22/sequential_16/activation_47/Relu, batch_normalization_182/gamma/read, model_22/sequential_16/batch_normalization_182/FusedBatchNorm:3, model_22/sequential_16/batch_normalization_182/FusedBatchNorm:4)]]

Caused by op 'training_10/Adadelta/gradients/model_22/sequential_16/batch_normalization_182/FusedBatchNorm_grad/FusedBatchNormGrad', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/

In [ ]:
    context_encoder.save_model()

In [ ]:
    (_, _), (X_test, y_test) = data
    idx = [278, 451, 2354]
    img_in, img_out = X_test[idx], y_test[idx]
    context_encoder.sample_maps(50000, img_in, img_out)

In [ ]:
(_, _), (X_test, y_test) = data

In [ ]:
X_test_pred = context_encoder.predict(X_test)
y_test_bi = np.bool8(y_test > 0)  # -1 - 1
X_test_pred_bi = np.bool8(X_test_pred > 0)

X_test_pred_bi = np.reshape(X_test_pred_bi, (-1, 128, 128))
y_test_bi = np.reshape(y_test_bi, (-1, 128, 128))
accs = [accuracy_score(i.flatten(), j.flatten()) for i, j in zip(X_test_pred_bi, y_test_bi)]
print(np.mean(accs))